# Optimal Trader Joe's Grocery List

In [11]:
!pip install pulp --quiet

In [12]:
import pandas as pd
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpStatus

## Loading Data

In [ ]:
data = pd.read_csv('data/nutrition_prices.csv')
data.head()

<ipython-input-8-eff41bcabf5e>:1: DtypeWarning: Columns (2,10,11,18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/nutrition_prices.csv')


,Unnamed: 0.1,Unnamed: 0,serving_size,calories,total_fat,saturated_fat,trans_fat,cholesterol,sodium,total_carbohydrates,dietary_fiber,sugars,protein,vitamin_d,calcium,iron,potassium,item,item_title,retail_price
0,0,0,2 tbsp. (28g),70.0,7.0,5.0,0.0,20.0,70.0,1.0,0,0.5,2.0,0.0,52.0,0.0,0,"""Stracciatella"" Burata Filling",Burrata Filling,4.49
1,225,225,0.8 cup (170g),110.0,0.0,0.0,0.0,10.0,75.0,7.0,0,5,17.0,0.0,190.0,0.0,240,"0% Greek Yogurt, Nonfat, Plain",Nonfat Plain Greek Yogurt,0.99
2,818,818,1 container (150g),130.0,0.0,0.0,0.0,5.0,60.0,18.0,0,15,11.0,0.8,130.0,0.4,188,0% Milkfat Greek Nonfat Yogurt,Greek Spanakopita,4.49
3,1407,1407,1 container (150g),130.0,0.0,0.0,0.0,5.0,60.0,18.0,0,15,12.0,0.8,130.0,0.4,188,0% Milkfat Strawberry Greek Nonfat Yogurt,Sparkling Strawberry Juice,3.99
4,1993,1993,1 1-inch cube (28g),120.0,10.0,7.0,0.0,30.0,270.0,0.0,0,0,8.0,0.0,260.0,0.0,0,"1,000 Day Gouda Cheese",1000 Day Gouda Cheese,12.99


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59024 entries, 0 to 59023
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0.1         59024 non-null  int64  
 1   Unnamed: 0           27772 non-null  object 
 2   serving_size         27702 non-null  object 
 3   calories             58953 non-null  float64
 4   total_fat            58430 non-null  float64
 5   saturated_fat        58415 non-null  float64
 6   trans_fat            58412 non-null  float64
 7   cholesterol          59004 non-null  float64
 8   sodium               59021 non-null  float64
 9   total_carbohydrates  58429 non-null  float64
 10  dietary_fiber        58415 non-null  object 
 11  sugars               59013 non-null  object 
 12  protein              22707 non-null  float64
 13  vitamin_d            23205 non-null  float64
 14  calcium              23204 non-null  float64
 15  iron                 32473 non-null 

## Cleaning Data

In [14]:
# drop useless cols
columns_to_drop = ['Unnamed: 0.1', 'Unnamed: 0', 'item_title' ] # 'item' is the correct name column, 'item_title' has errors
nutrition_data_cleaned = data.drop(columns=columns_to_drop)

# Cols to numeric
numeric_columns = ['dietary_fiber', 'sugars', 'potassium']
for col in numeric_columns:
    nutrition_data_cleaned[col] = pd.to_numeric(nutrition_data_cleaned[col], errors='coerce')

In [15]:
# New Preprocessing steps - drop only critical missing values, drop duplicates, impute missing values
nutrition_data_cleaned = nutrition_data_cleaned.drop_duplicates(['item']) ## Drop Dups
nutrition_data_cleaned = nutrition_data_cleaned.dropna(subset=['item'])

# set missing serving size as Unknown
nutrition_data_cleaned['serving_size'] = nutrition_data_cleaned['serving_size'].fillna('Unknown')
# set missing  macros, micros to zero (NaN are almost exclusively used when that value is truly zero)
nutrition_data_cleaned.loc[:, [col for col in nutrition_data_cleaned.columns if col not in  ['calories', 'retail_price']]] = nutrition_data_cleaned.loc[:, [col for col in nutrition_data_cleaned.columns if col != 'calories']].fillna(0)

# calculating Calories from macros - 9 cal per gram of Fat, 4 cal per gram of protein, carb
nutrition_data_cleaned["calc_calories"] = nutrition_data_cleaned['total_fat']*9 + (nutrition_data_cleaned['total_carbohydrates'] + nutrition_data_cleaned['protein'])*4
nutrition_data_cleaned['calories'] = nutrition_data_cleaned['calories'].fillna(nutrition_data_cleaned["calc_calories"])  # replace missing calorie values with calculated values

# Replacing missing price values using additional dataset - traderjoesprices.com
latest_prices = pd.read_csv('data/traderjoes-dump.csv',skiprows=2, names=['sku','latest_price','item','date','store_cod','avaiability']).drop_duplicates(['item']).dropna()
nutrition_data_cleaned = nutrition_data_cleaned.merge(latest_prices[['item','latest_price']], how='left', on='item')
nutrition_data_cleaned['retail_price'] = nutrition_data_cleaned['retail_price'].fillna(nutrition_data_cleaned["latest_price"])  # replace missing calorie values with calculated values
nutrition_data_cleaned = nutrition_data_cleaned.drop(columns=['latest_price', 'calc_calories'])
nutrition_data_cleaned.dropna(inplace=True) # drop any rows that have not be cleaned fully

# Add fruits (not present our dataset)
nutrition_data_cleaned['fresh'] = 0.0
fruits = pd.read_csv('data/fruits.csv')
fruits['serving_size'] = fruits['serving_size'].astype('object')
fruits['fresh'] = 1.0

nutrition_data_cleaned = pd.concat([nutrition_data_cleaned, fruits], ignore_index=True)
nutrition_data_cleaned = nutrition_data_cleaned.drop_duplicates(subset='item', keep="last").reset_index().drop(columns=['index'])

# drop unreasonable values
nutrition_data_cleaned = nutrition_data_cleaned[
(nutrition_data_cleaned['calories'] >= 0) &
(nutrition_data_cleaned['calories'] < 2000) &
(nutrition_data_cleaned['retail_price'] > 0.01)
]

nutrition_data_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
Index: 202 entries, 0 to 242
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   serving_size         202 non-null    object 
 1   calories             202 non-null    float64
 2   total_fat            202 non-null    float64
 3   saturated_fat        202 non-null    float64
 4   trans_fat            202 non-null    float64
 5   cholesterol          202 non-null    float64
 6   sodium               202 non-null    float64
 7   total_carbohydrates  202 non-null    float64
 8   dietary_fiber        202 non-null    float64
 9   sugars               202 non-null    float64
 10  protein              202 non-null    float64
 11  vitamin_d            202 non-null    float64
 12  calcium              202 non-null    float64
 13  iron                 202 non-null    float64
 14  potassium            202 non-null    float64
 15  item                 202 non-null    object 


In [16]:
# export data
nutrition_data_cleaned.to_csv('cleaned_data_latest.csv', index=False)

## Optimization Model

#### Model

In [27]:
model = LpProblem("Optimal_Grocery_List", LpMinimize)

#### Binary Decision Variables

In [28]:
x = {i: LpVariable(f"x_{i}", lowBound = 0, upBound=3, cat="Integer") for i in nutrition_data_cleaned.index}

#### Objective Function to Minimize Cost

In [29]:
model += lpSum(nutrition_data_cleaned.loc[i, 'retail_price'] * x[i] for i in nutrition_data_cleaned.index), "Total Cost"

#### Constraints

In [30]:
# daily intake
constraints = {
    "calories": 2200,
    "protein": 60,
    "fat_max": 80,
    "carbohydrate": 250,
    "sodium_max": 3000,
    "fiber": 20,
    "sugar_max": 60,
    "cholesterol_max": 300,
    "saturated_fat_max": 20,
    "vitamin_d": 10,
    "budget": 100,
    'fruits_min':7.0
}

In [31]:
# Constraint 1: Calories
model += lpSum(nutrition_data_cleaned.loc[i, 'calories'] * x[i] for i in nutrition_data_cleaned.index) >= constraints["calories"]*7, "Calorie_Constraint"
# Constraint 2: Protein
model += lpSum(nutrition_data_cleaned.loc[i, 'protein'] * x[i] for i in nutrition_data_cleaned.index) >= constraints["protein"]*7, "Protein_Constraint"
# Constraint 3: Fat
model += lpSum(nutrition_data_cleaned.loc[i, 'total_fat'] * x[i] for i in nutrition_data_cleaned.index) <= constraints["fat_max"]*7, "Fat_Constraint"
# Constraint 4: Carbohydrates
model += lpSum(nutrition_data_cleaned.loc[i, 'total_carbohydrates'] * x[i] for i in nutrition_data_cleaned.index) >= constraints["carbohydrate"]*7, "Carbohydrate_Constraint"
# Constraint 5: Sodium
model += lpSum(nutrition_data_cleaned.loc[i, 'sodium'] * x[i] for i in nutrition_data_cleaned.index) <= constraints["sodium_max"]*7, "Sodium_Constraint"
# Constraint 6: Fiber
model += lpSum(nutrition_data_cleaned.loc[i, 'dietary_fiber'] * x[i] for i in nutrition_data_cleaned.index) >= constraints["fiber"]*7, "Fiber_Constraint"
# Constraint 7: Sugar
model += lpSum(nutrition_data_cleaned.loc[i, 'sugars'] * x[i] for i in nutrition_data_cleaned.index) <= constraints["sugar_max"]*7, "Sugar_Constraint"
# Constraint 8: Cholesterol
model += lpSum(nutrition_data_cleaned.loc[i, 'cholesterol'] * x[i] for i in nutrition_data_cleaned.index) <= constraints["cholesterol_max"]*7, "Cholesterol_Constraint"
# Constraint 9: Saturated Fat
model += lpSum(nutrition_data_cleaned.loc[i, 'saturated_fat'] * x[i] for i in nutrition_data_cleaned.index) <= constraints["saturated_fat_max"]*7, "Saturated_Fat_Constraint"
# Constraint 10: Vitamin D
model += lpSum(nutrition_data_cleaned.loc[i, 'vitamin_d'] * x[i] for i in nutrition_data_cleaned.index) >= constraints["vitamin_d"]*7, "Vitamin_D_Constraint"
# Constraint 11: Budget
model += lpSum(nutrition_data_cleaned.loc[i, 'retail_price'] * x[i] for i in nutrition_data_cleaned.index) <= constraints["budget"], "Budget_Constraint"
# Constraint 11: Fruits
model += lpSum(nutrition_data_cleaned.loc[i, 'fresh'] * x[i] for i in nutrition_data_cleaned.index) >= constraints["fruits_min"], "Fruits_Constraint"

In [ ]:
# export model
model.writeMPS("model_file.mps");

#### Model Solver

In [32]:
model.solve()
print("Model Status:", LpStatus[model.status])

Model Status: Optimal


In [36]:
# Print optimal item set
selected_items = [nutrition_data_cleaned.loc[i, 'item'] + f' x {x[i].value()}'  for i in nutrition_data_cleaned.index if x[i].value() >= 1]
print("Selected Items:")
for item in selected_items:
    print(item)

total_cost = sum(nutrition_data_cleaned.loc[i, 'retail_price'] * x[i].value() for i in nutrition_data_cleaned.index if x[i].value() >= 1)
print(f"\nTotal Cost: ${total_cost:.2f}")

Selected Items:
0% Greek Yogurt, Nonfat, Plain x 3.0
100% Pure Florida Grapefruit Juice, Ruby Red x 3.0
12 Grain Mini Snack Crackers x 3.0
90% Lean 10% Fat Heirloom Ground Chicken x 3.0
Alaskan Wild Sockeye Salmon Fillet Portions x 3.0
Albacore Tuna x 3.0
Albacore Tuna in water x 3.0
All Natural Boneless Skinless Chicken Thighs x 1.0
Almond Butter Chia Overnight Oats x 3.0
Ancient Grain & Super Seed Oatmeal x 2.0
Angus Beef Chili, with Pinto Beans x 1.0
Chocolate Covered Wafer Cookie with Peanut Butter Filling x 1.0
The Dark Chocolate Lover's Chocolate Bar x 1.0
Apricot x 1.0
Banana x 3.0
Passion fruit x 3.0

Total Cost: $72.78


Top K Solutions (Solution Pool)

In [26]:
# Loop to generate TOP K optimal solutions, for sake of introducing variety to meal plans
import os

grocery_lists = open('grocery_lists.txt','w')
iter = 0
K = 5 # fetch top K solutions
while True:
    model.solve()
    # The solution is printed if it was deemed "optimal" i.e met the constraints
    if LpStatus[model.status] == "Optimal":
        selected_items_dict = {nutrition_data_cleaned.loc[i, 'item']: x[i].value() for i in nutrition_data_cleaned.index if x[i].value() >= 1}
        selected_items = list(selected_items_dict.keys())
        selected_qty = list(selected_items_dict.values())
        selected_items_with_qty = [selected_items[i] + ' x ' + str(selected_qty[i]) for i in range(len(selected_items))]

        if iter == 0:
          optimal_item_list = selected_items # store the optimal grocery list to compare alternatives with
        total_cost = sum(nutrition_data_cleaned.loc[i, 'retail_price'] * x[i].value() for i in nutrition_data_cleaned.index if x[i].value() >= 1)
        # Macros
        Calories = sum(nutrition_data_cleaned.loc[i, 'calories'] * x[i].value() for i in nutrition_data_cleaned.index if x[i].value() >= 1)
        Protein = sum(nutrition_data_cleaned.loc[i, 'protein'] * x[i].value() for i in nutrition_data_cleaned.index if x[i].value() >= 1)
        Fat = sum(nutrition_data_cleaned.loc[i, 'total_fat'] * x[i].value() for i in nutrition_data_cleaned.index if x[i].value() >= 1)
        Carbohydrates = sum(nutrition_data_cleaned.loc[i, 'total_carbohydrates'] * x[i].value() for i in nutrition_data_cleaned.index if x[i].value() >= 1)
        macros = f'MACROS: Calories:{Calories:.0f} | Protein:{Protein:.0f} | Fat:{Fat:.0f} | Carbs:{Carbohydrates:.0f}'
        # Write solution to the grocery_lists.txt file
        if iter == 0:
          title = 'OPTIMAL GROCERY LIST'
          optimal_list = f'{title}: Total Cost: ${total_cost:.2f}\n' + f'{macros} \n-' + '\n-'.join(selected_items_with_qty) + '\n\n'

        else:
          title = f'ALTERNATIVE GROCERY LIST {iter}'
          removed_items = list(set(optimal_item_list) - set(selected_items))
          added_items = list(set(selected_items) - set(optimal_item_list))
          optimal_list = f'{title}: Total Cost: ${total_cost:.2f}\n' + f'{macros} \n-' + '\n-'.join(selected_items_with_qty) + f'\nItems removed: {removed_items}'+ f'\nItems added: {added_items}' + '\n\n'

        grocery_lists.write(optimal_list)
        print(optimal_list)

        try: # delete existing constraint if exists
          del model.constraints['OptimalSol']
        except:
          pass
        model += lpSum(nutrition_data_cleaned.loc[i, 'retail_price'] * x[i] for i in nutrition_data_cleaned.index) >= total_cost+1.0, f"OptimalSol"
        iter +=1
        if iter >= K: # only get top K
          try: # delete existing constraint if exists
            del model.constraints['OptimalSol']
          except:
            pass
          break
    # If a new optimal solution cannot be found, we end the program
    else:
        break

grocery_lists.close()

# The location of the solutions is give to the user
print("Solutions Written to grocery_lists.txt")

OPTIMAL GROCERY LIST: Total Cost: $72.78
MACROS: Calories:15798 | Protein:420 | Fat:532 | Carbs:2543 
-0% Greek Yogurt, Nonfat, Plain x 3.0
-100% Pure Florida Grapefruit Juice, Ruby Red x 3.0
-12 Grain Mini Snack Crackers x 3.0
-90% Lean 10% Fat Heirloom Ground Chicken x 3.0
-Alaskan Wild Sockeye Salmon Fillet Portions x 3.0
-Albacore Tuna x 3.0
-Albacore Tuna in water x 3.0
-All Natural Boneless Skinless Chicken Thighs x 1.0
-Almond Butter Chia Overnight Oats x 3.0
-Ancient Grain & Super Seed Oatmeal x 2.0
-Angus Beef Chili, with Pinto Beans x 1.0
-Chocolate Covered Wafer Cookie with Peanut Butter Filling x 1.0
-The Dark Chocolate Lover's Chocolate Bar x 1.0
-Apricot x 1.0
-Banana x 3.0
-Passion fruit x 3.0


ALTERNATIVE GROCERY LIST 1: Total Cost: $73.78
MACROS: Calories:15402 | Protein:420 | Fat:559 | Carbs:2187 
-0% Greek Yogurt, Nonfat, Plain x 3.0
-100% Pure Florida Grapefruit Juice, Ruby Red x 3.0
-12 Grain Mini Snack Crackers x 3.0
-90% Lean 10% Fat Heirloom Ground Chicken x 2.

#### Sensitivity Analysis: Shadow Prices and Slack

In [34]:
# Sensitivity Analysis: Shadow Prices and Slack
print("\nSensitivity Analysis:")
for constraint_name, constraint in model.constraints.items():
    print(f"Constraint: {constraint_name}")
    print(f"  Shadow Price (Dual Value): {constraint.pi}")
    print(f"  Slack: {constraint.slack}")




Sensitivity Analysis:
Constraint: Calorie_Constraint
  Shadow Price (Dual Value): 0.0
  Slack: -398.0
Constraint: Protein_Constraint
  Shadow Price (Dual Value): 0.0
  Slack: -0.2699999999999818
Constraint: Fat_Constraint
  Shadow Price (Dual Value): 0.0
  Slack: 27.940000000000055
Constraint: Carbohydrate_Constraint
  Shadow Price (Dual Value): 0.0
  Slack: -792.7800000000002
Constraint: Sodium_Constraint
  Shadow Price (Dual Value): 0.0
  Slack: 6745.0
Constraint: Fiber_Constraint
  Shadow Price (Dual Value): 0.0
  Slack: -1717.0
Constraint: Sugar_Constraint
  Shadow Price (Dual Value): 0.0
  Slack: 213.47
Constraint: Cholesterol_Constraint
  Shadow Price (Dual Value): 0.0
  Slack: 91.0
Constraint: Saturated_Fat_Constraint
  Shadow Price (Dual Value): 0.0
  Slack: 80.91
Constraint: Vitamin_D_Constraint
  Shadow Price (Dual Value): 0.0
  Slack: -21.599999999999994
Constraint: Fruits_Constraint
  Shadow Price (Dual Value): 0.0
  Slack: -0.0


INTERACTIVE UI

In [ ]:
!pip install streamlit --quiet
!pip install pulp --quiet
!npm install localtunnel  --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 52.2 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 3s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [ ]:
%%writefile TJGroceryStreamlit.py

import streamlit as st
import pandas as pd
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpStatus
import matplotlib.pyplot as plt
from google.colab import files as FILE
import os
import requests
import time
import streamlit as st

# Download TJ's logo
img_data = requests.get('https://cdn.worldvectorlogo.com/logos/trader-joes-logo.svg').content # https://1000logos.net/wp-content/uploads/2022/03/Trader-Joes-Logo.png
with open('logo.svg', 'wb') as handler:
    handler.write(img_data)

st.image('logo.svg')
st.title("Optimal Trader Joe's Grocery List")

st.write("Welcome to the Trader Joe's Grocery List Optimizer. Use the sidebar to set your preferences.")

st.sidebar.header("Set Your Preferences")

st.sidebar.subheader("Weekly Preferences")
# Budget input
Budget_Preference = st.sidebar.number_input("Enter your weekly budget ($)", min_value=0, value=150, step=5)
# Broad Preferences
Repetition_Preference = st.sidebar.number_input("Enter maximum quantity per item selected (weekly)", min_value=0, value=3, step=1)
Fruit_Preference = st.sidebar.number_input("Enter minimum types of fresh fruit included (weekly)", min_value=0, value=7, step=1)
Alternatives_Preference = st.sidebar.number_input("Enter number of item alternatives/substitutions requested", min_value=0, value=5, step=1)

# Macro Preferences
st.sidebar.subheader("Daily Macro Preferences")
Calorie_Preference = st.sidebar.number_input("Enter your daily calorie Preference", min_value=0, value=2200, step=100)
Protein_Preference = st.sidebar.number_input("Min. Protein (g)", min_value=0, value=60, step=5)
Carbohydrate_Preference = st.sidebar.number_input("Min. Carbohydrates (g)", min_value=0, value=250, step=10)
Fat_Preference = st.sidebar.number_input("Max. Fat (g)", min_value=0, value=80, step=5)

# Micro Preferences
st.sidebar.subheader("Daily Micro Preferences")
Sodium_Preference = st.sidebar.number_input("Max. Sodium (mg)", min_value=0, value=3000, step=100)
Fiber_Preference = st.sidebar.number_input("Min. Fiber (g)", min_value=0, value=20, step=1)
Sugar_Preference = st.sidebar.number_input("Max. Sugar (g)", min_value=0, value=60, step=1)
Cholesterol_Preference = st.sidebar.number_input("Max. Cholesterol (mg)", min_value=0, value=300, step=10)
Saturated_Fat_Preference = st.sidebar.number_input("Max. Saturated Fat (g)", min_value=0, value=25, step=1)
Vitamin_D_Preference = st.sidebar.number_input("Min. Vitamin D (mcg)", min_value=0, value=15, step=1)

preferences_dict = {'Calorie':Calorie_Preference, 'Protein':Protein_Preference, 'Fat':Fat_Preference, 'Carbohydrate':Carbohydrate_Preference, 'Sodium':Sodium_Preference,
                        'Fiber':Fiber_Preference,'Sugar':Sugar_Preference, 'Cholesterol':Cholesterol_Preference,'Saturated_Fat':Saturated_Fat_Preference,
                    'Vitamin_D':Vitamin_D_Preference, 'Repetition':Repetition_Preference, 'Fruit':Fruit_Preference, 'Alternatives':Alternatives_Preference}


# import TJs data

@st.cache_data
def load_df():
    return pd.read_csv('cleaned_data_latest.csv')

nutrition_data_cleaned = load_df()


def optimize_grocery_list(data=nutrition_data_cleaned, preferences_dict=preferences_dict):

    # creating  model
    uimodel = LpProblem("Optimal_Grocery_List", LpMinimize)
    x = {i: LpVariable(f"x_{i}", lowBound = 0, upBound=preferences_dict['Repetition'], cat="Integer") for i in nutrition_data_cleaned.index}
    # creating constraints

    # weekly values
    uimodel += lpSum(nutrition_data_cleaned.loc[i, 'retail_price'] * x[i] for i in nutrition_data_cleaned.index), "Total Cost"
    uimodel += lpSum(nutrition_data_cleaned.loc[i, 'retail_price'] * x[i] for i in nutrition_data_cleaned.index) <= Budget_Preference, "Budget_Constraint"
    uimodel += lpSum(nutrition_data_cleaned.loc[i, 'fresh'] * x[i] for i in nutrition_data_cleaned.index) >= Fruit_Preference, "Fruit_Constraint"



    column_dict = {'Calorie':'calories', 'Protein':'protein', 'Fat':'total_fat', 'Carbohydrate':'total_carbohydrates', 'Sodium':'sodium','Fiber':'dietary_fiber','Sugar':'sugars',
                          'Cholesterol':'cholesterol','Saturated_Fat':'saturated_fat','Vitamin_D':'vitamin_d', 'Fruit':'fresh'}

    for item in list(column_dict.keys()):
        if item in ['Fat', 'Sodium', 'Sugar', 'Cholesterol', 'Saturated_Fat']: # max
          uimodel += lpSum(nutrition_data_cleaned.loc[i, column_dict[item]] * x[i] for i in nutrition_data_cleaned.index) <= preferences_dict[item]*7, f"{item}_Constraint"
        if item in ['Calorie',  'Protein',  'Carbohydrate',  'Fiber',  'Vitamin_D']: # min
          uimodel += lpSum(nutrition_data_cleaned.loc[i, column_dict[item]] * x[i] for i in nutrition_data_cleaned.index) >= preferences_dict[item]*7, f"{item}_Constraint"    # solving on click

    # Run Model
    uimodel.solve()
    print("Model Status:", LpStatus[uimodel.status])
    optimal_items_dict = {nutrition_data_cleaned.loc[i, 'item']: x[i].value() for i in nutrition_data_cleaned.index if x[i].value() >= 1}
    optimal_items = list(optimal_items_dict.keys())
    optimal_qty = list(optimal_items_dict.values())
    optimal_items_with_qty = [optimal_items[i] + ' x ' + str(optimal_qty[i]) for i in range(len(optimal_items))]
    # Macros
    Calories = sum(nutrition_data_cleaned.loc[i, 'calories'] * x[i].value() for i in nutrition_data_cleaned.index if x[i].value() >= 1)
    Protein = sum(nutrition_data_cleaned.loc[i, 'protein'] * x[i].value() for i in nutrition_data_cleaned.index if x[i].value() >= 1)
    Fat = sum(nutrition_data_cleaned.loc[i, 'total_fat'] * x[i].value() for i in nutrition_data_cleaned.index if x[i].value() >= 1)
    Carbohydrates = sum(nutrition_data_cleaned.loc[i, 'total_carbohydrates'] * x[i].value() for i in nutrition_data_cleaned.index if x[i].value() >= 1)
    macros = [Calories, Protein,Fat,Carbohydrates]
    total_cost = sum(nutrition_data_cleaned.loc[i, 'retail_price'] * x[i].value() for i in nutrition_data_cleaned.index if x[i].value() >= 1)

    # Generate Alternatives
    K = preferences_dict['Alternatives'] # fetch top K solutions
    iter = 0
    removed_items = []
    added_items = []
    while True:
        try: # delete existing constraint if exists
          del uimodel.constraints['OptimalSol']
        except:
          pass
        uimodel += lpSum(nutrition_data_cleaned.loc[i, 'retail_price'] * x[i] for i in nutrition_data_cleaned.index) >= total_cost+iter+1.0, f"OptimalSol"
        uimodel.solve()
        # The solution is printed if it was deemed "optimal" i.e met the constraints
        if LpStatus[uimodel.status] == "Optimal":
            selected_items_dict = {nutrition_data_cleaned.loc[i, 'item']: x[i].value() for i in nutrition_data_cleaned.index if x[i].value() >= 1}
            selected_items = list(selected_items_dict.keys())
            selected_qty = list(selected_items_dict.values())
            selected_items_with_qty = [selected_items[i] + ' x ' + str(selected_qty[i]) for i in range(len(selected_items))]

            removed_items.append(list(set(optimal_items) - set(selected_items)))
            added_items.append(list(set(selected_items) - set(optimal_items)))

            iter +=1
            if iter >= K: # only get top K
              try: # delete existing constraint if exists
                del uimodel.constraints['OptimalSol']
              except:
                pass
              break
        # If a new optimal solution cannot be found, we end the program
        else:
            break
    alternatives_df = pd.DataFrame({'Remove these items':removed_items,'Add these items':added_items})

    return optimal_items_dict, macros, total_cost, alternatives_df

# Displaying Results

if st.button("Optimize Grocery List"):

    selected_items_dict, macros, total_cost, alternatives_df =  optimize_grocery_list()
    with st.status("Generating Optimal Grocery List", expanded=True):
      st.write("Defining Model...")
      time.sleep(2)
      st.write("Solving Model...")
      time.sleep(3)
      st.write("Generating Alternatives...")
      time.sleep(5)

    st.success("Optimization complete!")
    st.subheader("Optimal Grocery List")

    selected_items = list(selected_items_dict.keys())
    selected_qty = list(selected_items_dict.values())
    selected_items_df = pd.DataFrame({'Item':selected_items, 'Qty':selected_qty})

    matching_rows = nutrition_data_cleaned[nutrition_data_cleaned['item'].isin(selected_items)]
    try:

      # retrieve item w/ features from dataframe
      display_rows = matching_rows.drop(columns=['iron', 'calcium', 'potassium', 'fresh'], )
      display_rows = display_rows.rename(columns={'serving_size':'Serving Size'	, 'calories':'Calories',	'total_fat':'Total Fat',	'saturated_fat':"Saturated Fat",	'trans_fat':'Trans Fat',
                      'cholesterol':'Cholesterol',	'sodium':'Sodium',	'total_carbohydrates':'Carbs',	'dietary_fiber':'Fiber',	'sugars':'Sugar',
                      "protein":'Protein',	'vitamin_d':"Vitamin D", 'item':'Item', 'retail_price':'Price'})
      display_rows = display_rows.merge(selected_items_df, how='left', on='Item')
      col_order = ['Item', 'Qty' ,'Price', 'Serving Size', 'Calories', 'Carbs','Protein', 'Total Fat', 'Saturated Fat', 'Cholesterol',
                'Sodium',  'Fiber', 'Sugar', 'Vitamin D', ]
      display_rows = display_rows[col_order]
      st.dataframe(display_rows)
    except:
      st.dataframe(matching_rows)


    st.subheader(f"Total Cost: ${total_cost:.2f} | Total Calories - {macros[0]:.0f}")
    st.subheader(f"MACROS: Protein - {macros[1]:.0f} | Fat - {macros[2]:.0f} | Carbs - {macros[3]:.0f}")

    st.subheader("Substitutions")
    st.write("""Swap these items from the generated list with any set of alternative to create a meal plan more suited to your taste.
    Alternatives are nearly identical in Macros, and only $1-4 more expensive than the original list.""")
    st.dataframe(alternatives_df)


    # Visualize the results
    st.subheader("Macronutrient Distribution")
    nutrient_data = pd.DataFrame({
        'Nutrient': ['Protein', 'Fat', 'Carbohydrates'],
        'Amount': [macros[1], macros[2], macros[3]]
    })

    # Pie chart, Macros:
    macronutrient_data = pd.DataFrame({
            'Nutrient': ['Protein', 'Fat', 'Carbohydrates'],
            'Amount': [macros[1], macros[2], macros[3]]
        })

    explode = (0.1, 0, 0.1)  # only "explode" the protein and carb slices

    fig1, ax1 = plt.subplots()
    ax1.pie(macronutrient_data['Amount'].to_list(), explode=explode, labels=macronutrient_data['Nutrient'].to_list(),
            shadow=True, startangle=90, autopct='%1.1f%%')
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

    st.pyplot(fig1)

    st.subheader("Micronutrient Distribution (mg)")

    # Pie chart, Micros:
    micronutrient_data = pd.DataFrame({
            'Nutrient':['Cholesterol','Sodium','Vitamin D', "Calcium",'Iron','Potassium'],
            'Amount':  matching_rows[['cholesterol','sodium','vitamin_d','calcium','iron','potassium']].sum().values
        })

    explode = (0.1, 0.1, 0 ,0,0,0)  # only "explode" the chol and sodium slices

    fig2, ax2 = plt.subplots()
    ax2.pie(micronutrient_data['Amount'].to_list(), labels=micronutrient_data['Nutrient'].to_list(),
            shadow=True, startangle=90, autopct='%d')
    ax2.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

    st.pyplot(fig2)



Overwriting TJGroceryStreamlit.py


In [ ]:
!curl ipecho.net/plain
!streamlit run TJGroceryStreamlit.py &>/content/logs.txt &
!npx localtunnel --port 8501


34.75.78.208⠙your url is: https://whole-clocks-wave.loca.lt
^C
